# Exploratory Data Analysis

# A. Setup

In [ ]:
#importing relevant packages
%run /Users/thomasadler/Desktop/futuristic-platipus/notebooks/ta_01_packages.py

In [ ]:
# import useful functions
%run /Users/thomasadler/Desktop/futuristic-platipus/notebooks/ta_02_functions.py

In [ ]:
#defining working directory
filepath = '/Users/thomasadler/Desktop/capstone_docs/'

In [ ]:
#data dictionary part A
Image("/Users/thomasadler/Desktop/futuristic-platipus/data_dictionary/4A-Master-Dictionary.png")

In [ ]:
#data dictionary part B
Image("/Users/thomasadler/Desktop/futuristic-platipus/data_dictionary/4B-Master-Dictionary.png")

Now we have a clean dataset, with no duplicate rows/columns, no missing values, all of our columns of interest and all of them in a format fit for analysis. Our outcome (dependent) variable is whether a water point is functioning or not. `is_functioning` is a binary column equal to 1 if that water point was functioning at the time of check, 0 if not.

In [ ]:
#water points
master_df_raw=pd.read_csv(filepath + 'master_df.csv')

#leaving raw dataset untouched

master_df=master_df_raw.copy()

#check
master_df.info()

In [ ]:
#dropping legacy index column
master_df.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
#summary statistics
round(master_df.describe().T)

We will go through every column, and understand the information it contains and how we can use it in our models. We will use the summary statistic table above for every variable.

# B. Distribution of variables

## 1. wpdx_id

In [ ]:
#unique water points
unique_water=len(set(master_df['wpdx_id']))
total_observations=len(master_df['wpdx_id'])

print(f"There are {unique_water} unique water points in the dataset.")
print(f"There are {total_observations} reports in the dataset.")
print(f"There are {total_observations-unique_water} water points with more than one report.")

In [ ]:
#reports by water point
reports_water_pt=master_df[['wpdx_id','clean_adm1' ]].groupby('wpdx_id').count()

#visualise
sns.histplot(reports_water_pt)

plt.title("Majority of water points have only been checked once")

plt.xlabel("Number of reports")

plt.legend()

plt.show()

#percentages
round(reports_water_pt.value_counts(normalize=True)*100,2)

## 2-3. lat_deg & lon_deg

In [ ]:
#location of all water points
unique_water_points=master_df.groupby('wpdx_id').mean()

In [ ]:
# #visualise water points, choose what variable represents the size of the points
# fig = px.scatter_geo(
#     water_points,
#     lon='lon_deg', lat='lat_deg', 
#     size='served_population', #'crucialness', 'pressure', 'total_fatalities_adm4', 'total_events_adm4 
#     height=600,
#     width=800,
# )

# fig.show()

## 4. is_functioning

In [ ]:
#functioning water points
func_distrib=master_df['is_functioning'].value_counts(normalize=True)*100

print(
    f"The distribution of water points is {round(func_distrib[0],0).astype('int')}% not functioning and {round(func_distrib[1],0).astype('int')}% functioning."
)

In [ ]:
#list of regional level
regions=['clean_adm2', 'clean_adm3', 'clean_adm4']

#visualise through a subplot
plt.subplots(2,2, figsize=(30,20))

for i, adm in enumerate(regions, 1):
    adm_functioning=master_df[[adm,'is_functioning']].groupby(adm).mean()*100
    plt.subplot(3,2,i)

    sns.histplot(adm_functioning)

    plt.xlabel(f"Proportion of water points functioning in {adm}", size=25)
    plt.ylabel('Count', size=20)
    plt.yticks(fontsize=20)
    plt.xticks(fontsize=20)

    plt.axvline(adm_functioning['is_functioning'].median(), c='gold', label='median')

    plt.legend()   
plt.tight_layout()
plt.show()
    


## 5-8. clean_adm

In [ ]:
#number of regions
for regions in ['clean_adm1', 'clean_adm2', 'clean_adm3', 'clean_adm4']:   
    print(f"There are {len(set(master_df[regions]))} {regions} regions in our Uganda dataset")

In [ ]:
#number of water point reports by region
adm1_reports=master_df[['clean_adm1', 'wpdx_id']].groupby('clean_adm1').count()

#visualise
sns.barplot(data=adm1_reports, y=adm1_reports.index, x=adm1_reports['wpdx_id'], palette="flare")

plt.xlabel('number of reports')

plt.axvline(adm1_reports['wpdx_id'].mean(), c='royalblue', label='mean')

plt.show()


In [ ]:
#list of regional level
regions=['clean_adm2', 'clean_adm3', 'clean_adm4']

#visualise through a subplot
plt.subplots(2,2, figsize=(30,20))

for i, adm in enumerate(regions, 1):
    adm_reports=master_df[[adm, 'wpdx_id']].groupby(adm).count()
    plt.subplot(3,1,i)

    sns.histplot(adm_reports)

    
    plt.xlabel(f"Number of reports by {adm}", size=25)
    plt.ylabel('Count', size=20)
    plt.yticks(fontsize=20)
    plt.xticks(fontsize=20)

    plt.axvline(adm_reports['wpdx_id'].median(), c='gold', label='median')  
    plt.axvline(adm_reports['wpdx_id'].mean(), c='r', label='mean')   

    plt.legend()
    plt.tight_layout()
plt.show()

## 9-13. distance_to...

In [ ]:
#visualise distances for water points
distances=['distance_to_primary', 'distance_to_secondary', 'distance_to_tertiary', 'distance_to_city', 'distance_to_town']

#creating subplot
plt.subplots(3,2, figsize=(30,20))

for i, distance in enumerate(distances, 1):
    plt.subplot(3,2,i)

    sns.histplot(unique_water_points[distance])

    plt.xlabel(f"{distance} for a water point", size=25)
    plt.ylabel('Count', size=20)
    plt.yticks(fontsize=20)
    plt.xticks(fontsize=20)

    plt.axvline(unique_water_points[distance].median(), c='gold', label='median')  
    plt.axvline(unique_water_points[distance].mean(), c='r', label='mean')    

    plt.legend()
    plt.tight_layout()
plt.show()

## 14. usage_cap

In [ ]:
#visualise
sns.distplot(unique_water_points['usage_cap'])

plt.xlabel('usage_cap', size=15)
plt.ylabel('Density', size=15)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)

plt.show()

#usage capacity
round((unique_water_points['usage_cap'].value_counts(normalize=True)*100).head(),2)

## 15. staleness_score

In [ ]:
#visualise
sns.distplot(unique_water_points['staleness_score'])

plt.xlabel('staleness_score', size=15)
plt.ylabel('Density', size=15)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.xlim(10,)

plt.axvline(unique_water_points['staleness_score'].mean(), c='r', label='mean')
plt.axvline(unique_water_points['staleness_score'].median(), c='gold', label='median') 

plt.legend()

plt.show()

## 16. cluster_size

In [ ]:
#proportion of each cluster size
unique_water_points['cluster_size'].value_counts(normalize=True).plot(kind='barh', ylabel='density', xlabel='cluster_size', fontsize=12)

#percentage
round(unique_water_points['cluster_size'].value_counts(normalize=True)*100,1)

## 17. is_complex_tech

In [ ]:
#proportion of water points with complex technology
tech_distrib=master_df['is_complex_tech'].value_counts(normalize=True)*100

print(
    f"The distribution of water points is {round(tech_distrib[0],0).astype('int')}% not complex technology and {round(tech_distrib[1],0).astype('int')}% complex technology."
)

## 18. is_installed_after_2006

In [ ]:
#proportion of water points installed after the year 2006
installed_distrib=master_df['is_installed_after_2006'].value_counts(normalize=True)*100

print(
    f"The distribution of water points is {round(installed_distrib[0],0).astype('int')}% installed after 2006 and {round(installed_distrib[1],0).astype('int')}% installed before 2006."
)

## 19. is_public_management

In [ ]:
#proportion of water points installed after the year 2006
public_distrib=master_df['is_public_management'].value_counts(normalize=True)*100

print(
    f"The distribution of water points is {round(public_distrib[0],0).astype('int')}% managed by public bodies and {round(public_distrib[1],0).astype('int')}% not managed by public bodies."
)

## 20. served_population

In [ ]:
#visualise served population
sns.histplot(unique_water_points['served_population'],)

plt.xlabel('served_population', size=15)
plt.ylabel('Count', size=15)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.xlim(0,1000)  
plt.ylim(0,10000)

plt.axvline(unique_water_points['served_population'].median(), c='gold', label='median')  
plt.axvline(unique_water_points['served_population'].mean(), c='r', label='mean')

plt.legend()

plt.show()

## 21. local_population

In [ ]:
#visualise local population
sns.histplot(unique_water_points['local_population'])

plt.xlabel('local_population', size=15)
plt.ylabel('Count', size=15)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.xlim(0,3000)  

plt.axvline(unique_water_points['local_population'].median(), c='gold', label='median')  
plt.axvline(unique_water_points['local_population'].mean(), c='r', label='mean')

plt.legend()

plt.show()

## 22. crucialness

In [ ]:
#visualise
sns.distplot(unique_water_points['crucialness'])

plt.xlabel('crucialness', size=15)
plt.ylabel('Density', size=15)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)

plt.axvline(unique_water_points['crucialness'].mean(), c='r', label='mean')
plt.axvline(unique_water_points['crucialness'].median(), c='gold', label='median') 

plt.legend()

plt.show()

## 23. pressure

In [ ]:
#visualise
sns.distplot(unique_water_points['pressure'])

plt.xlabel('pressure', size=15)
plt.ylabel('Density', size=15)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.xlim(0,10)

plt.axvline(unique_water_points['pressure'].mean(), c='r', label='mean')
plt.axvline(unique_water_points['pressure'].median(), c='gold', label='median') 

plt.legend()

plt.show()

## 24-45. Demographics and Regional statistics

In [ ]:
#visualise variables by adm1
adm1_df=master_df.groupby("clean_adm1").mean()

In [ ]:
#creating subplot
plt.subplots(22,2, figsize=(20,100))

#all variables by adm1 regional level
for i, variable in enumerate(adm1_df.columns, 1):
    plt.subplot(22,2,i)

    sns.barplot(data=adm1_df, y=adm1_df.index, x=adm1_df[variable], palette="Blues_d")

    plt.xlabel(f"average {variable}", size=16)
    plt.ylabel('adm1', size=16)
    plt.yticks(fontsize=14)
    plt.xticks(fontsize=14)

    plt.axvline(adm1_df[variable].mean(), c='r', label='mean')  
    plt.axvline(adm1_df[variable].median(), c='gold', label='median')

    plt.legend(loc='upper left')
    plt.tight_layout()

plt.show()

## 46. total_fatalities_adm4

In [ ]:
#dataset of fatalities by region
fatalities_adm=master_df[['clean_adm1', 'clean_adm2','clean_adm3', 'clean_adm4', 'total_fatalities_adm4']]\
    .groupby(['clean_adm1', 'clean_adm2','clean_adm3', 'clean_adm4']).mean()

fatalities_adm.reset_index(inplace=True)

#check
fatalities_adm.head()

In [ ]:
#list of regional level
regions=['clean_adm2', 'clean_adm3']

#visualise through a subplot
plt.subplots(1,2, figsize=(40,10))

for i, adm in enumerate(regions, 1):
    adm_fatalities=fatalities_adm[[adm,'total_fatalities_adm4']].groupby(adm).sum()
    plt.subplot(1,2,i)

    sns.histplot(adm_fatalities, binrange=(0,200), bins=10)

    plt.xlabel(f"Number of fatalities in each {adm}", size=30)
    plt.ylabel('Count', size=30)
    plt.yticks(fontsize=30)
    plt.xticks(fontsize=30)
    plt.ylim(0,)
    plt.xlim(0,300)

    plt.axvline(adm_fatalities['total_fatalities_adm4'].median(), c='gold', label='median')
    plt.axvline(adm_fatalities['total_fatalities_adm4'].mean(), c='r', label='mean')

    plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
#fatalities by adm4
adm4_fatalities=fatalities_adm[['clean_adm4','total_fatalities_adm4']].groupby('clean_adm4').mean()

sns.histplot(adm4_fatalities, binrange=(0,10), bins=10)
plt.xlabel(f"Number of fatalities in each adm4", size=10)
plt.ylabel('Count', size=10)
plt.yticks(fontsize=10)
plt.xticks(fontsize=10)

plt.axvline(adm4_fatalities['total_fatalities_adm4'].median(), c='gold', label='median')
plt.axvline(adm4_fatalities['total_fatalities_adm4'].mean(), c='r', label='mean')

plt.legend()
plt.show()

## 47. total_events_adm4

In [ ]:
#dataset of event by region
events_adm=master_df[['clean_adm1', 'clean_adm2','clean_adm3', 'clean_adm4', 'total_events_adm4']]\
    .groupby(['clean_adm1', 'clean_adm2','clean_adm3', 'clean_adm4']).mean()

events_adm.reset_index(inplace=True)

#check
events_adm.head()

In [ ]:
#list of regional level
regions=['clean_adm2', 'clean_adm3']

#visualise through a subplot
plt.subplots(1,2, figsize=(40,10))

for i, adm in enumerate(regions, 1):
    adm_events=events_adm[[adm,'total_events_adm4']].groupby(adm).sum()
    plt.subplot(1,2,i)

    sns.histplot(adm_events, binrange=(0,200), bins=10)

    plt.xlabel(f"Number of fatalities in each {adm}", size=30)
    plt.ylabel('Count', size=30)
    plt.yticks(fontsize=30)
    plt.xticks(fontsize=30)
    plt.ylim(0,)
    plt.xlim(0,300)

    plt.axvline(adm_events['total_events_adm4'].median(), c='gold', label='median')
    plt.axvline(adm_events['total_events_adm4'].mean(), c='r', label='mean')

    plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
#fatalities by adm4
adm4_events=events_adm[['clean_adm4','total_events_adm4']].groupby('clean_adm4').mean()

sns.histplot(adm4_events, binrange=(0,10), bins=10)
plt.xlabel(f"Number of fatalities in each adm4", size=10)
plt.ylabel('Count', size=10)
plt.yticks(fontsize=10)
plt.xticks(fontsize=10)

plt.axvline(adm4_events['total_events_adm4'].median(), c='gold', label='median')
plt.axvline(adm4_events['total_events_adm4'].mean(), c='r', label='mean')

plt.legend()
plt.show()

## 48. perc_local_served

In [ ]:
#visualise
sns.distplot(unique_water_points['perc_local_served'])

plt.xlabel('perc_local_served', size=15)
plt.ylabel('Density', size=15)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.xlim()

plt.axvline(unique_water_points['perc_local_served'].mean(), c='r', label='mean')
plt.axvline(unique_water_points['perc_local_served'].median(), c='gold', label='median') 

plt.legend()

plt.show()

# C. Correlation

In [ ]:
plt.figure(figsize=(80,80))

#Creating matrix of correlations between each pair of variables
matrix = np.triu(master_df.corr())

#Applying heatmap correlations
sns.heatmap(master_df.corr(), annot=True, mask=matrix, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation coefficients of each pair of variable', size=60)
plt.yticks(fontsize=25)
plt.xticks(fontsize=25)

plt.show()

The heatmap above is not easy to understand due to the number of columns. By zooming in we can identify a few trends:

- number of fatalities and number of events is heavily correlated which makes sense as the more conflicts the higher the chance of people getting hurt.
- crucialness and percentage of local population served also. They try to measure the same thing but in a slightly different way.
- the percentage of illiterate people is correlared with primary and secondary enrollment. People going to school will be able to read and write.
- more broadly, primary and secondary enrollment is heavily correlated with many other demographic variables
- mobile phone ownership, TV ownership, electricity access, bank account ownership are all similarly correlated with other demographic variable 

In [ ]:
# see only highly positively correlated variables
plt.figure(figsize=(80,80))

#Applying heatmap correlations
sns.heatmap(master_df.corr()>=0.7, annot=True, mask=matrix, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation coefficients of each pair of variable', size=60)
plt.yticks(fontsize=25)
plt.xticks(fontsize=25)

plt.show()

We look at variables which have a correlation coefficient higher than 0.7.

- we do not drop `perc_local_served` yet although it is very similar to what `crucialness`. It is also closely related to `pressure` as it is trying to measure the same thing. We will see later how to deal with those.

- we drop mobile phone and TV ownership in favor of electricity access and secondary enrollment as we can represent the two former with the two latter. In addition, we believe that electricity is a better overall indicator of the development and wealth of a household and region. Secondary enrollment gives us a good representation of the education system in the region.

- we drop the illiteracy rate and an indicator for households eating enough (less than 2 meals a day or not) in favour of access to toilet. The reason is that we already have school enrollment which should tell us already about the education situation of the region. In addition, toilet access also seems a strong candidate for a development indicator.

- we keep both events and fatalities. It makes sense they are highly correlated but will wait for further to analyse to choose which one to discard.

- it seems that households at a certain latitude live more often in temporary dwellings we also refrain from dropping it just yet as we might not be using latitude in most models.

In [ ]:
#drop columns from multicolinearity analysis
master_df_clean=master_df.drop(columns=['perc_hh_own_tv', 'perc_pop10p_mobile_phone', 'perc_pop18p_illiterate', 'perc_hh_less2meals'])

#check
master_df_clean.columns

In [ ]:
plt.figure(figsize=(80,80))

#Creating matrix of correlations between each pair of variables
matrix = np.triu(master_df_clean.corr())

#Applying heatmap correlations
sns.heatmap(master_df_clean.corr()<=-0.7, annot=True, mask=matrix, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation coefficients of each pair of variable', size=60)
plt.yticks(fontsize=25)
plt.xticks(fontsize=25)

plt.show()

We look at the variables which have a correlation coefficient larger than -0.8.
- we drop primary enrollment in favour of toilet access. We already have secondary enrollemnt to represent education level of a region.
- we drop subsistence farming in favour of electricity as we assume the latter is a better indicator for development 

In [ ]:
#drop columns from multicolinearity analysis
master_df_clean=master_df_clean.drop(columns=['perc_pop612_primary', 'perc_hh_subs_farm'])

#check
master_df_clean.columns

# D. Linearity with outcome variable

In [ ]:
#choose numeric columns
master_numeric=master_df_clean.select_dtypes(exclude='object')

In [ ]:
#checking the correlation coefficient between explanatory and outcome variable
correl_loop(master_numeric, master_numeric['is_functioning'])

Our correlation coefficients tell us that toilet access, secondary enrollment, bank account ownership and the fact that the water point was installed after 2006 is strongly correlated (larger than 0.5) with the functioning of a water point.

On the other hand, the percentage of households headed by a male in a region, cluster size, proportion of population in employment and fatalities in a region are not correlated with our outcome and may be dropped later down.

The function above loops over all variables in X and computes its correlation with y.
 
The hypotheses is:

$ 𝐻_0 $ : The variable X and y are not related (correlated), they are independent.

$ 𝐻_1 $ : X and y are related and not independent.

The output includes:

**Pearson correlation coefficient (PCC)**


* ex: If the PCC is 0.5 between x and y, that means that an increase of one unit in x is associated with an increase of 0.5 units for y.
 
 
**P-value**


* Represents the probability that we see our dataset, assuming $H_0$ is true.
     


**Positive or negative correlation between the two variables**


* $ -1 < PCC < -0.5 $ strong negative correlation
* $ -0.5 < PCC < 0 $ weak negative correlation
* $ 0 < PCC < 0.5 $ weak positive correlation
* $ 0.5 < PCC < 1 $ strong positive correlation


**Statistical significance of the PCC**

* If the $ p-value < 0.05 $  (5% significance level), then we can reject $ H_0 $. This means that the value we are testing for (PCC in this case) is statistically significant (**SS**) or not (**NSS**).

In [ ]:
#visualising linear relationship between all variables and our outcome variable
box_loop(master_numeric, master_numeric['is_functioning'])

The box plots do not tell us much more than the correlation coefficients from above apart from the fact that it sems that the majority of non-functioning water points are managed by a public entity.

# E. Variance

Knowint the variance of variables tells us whether they will be useful in our model later on. If variance is low then it won't have any variation to help us understand our outcome variable.

In [ ]:
#variance of numeric columns
master_numeric.var()

We need to scale our data as all columns have different scales and measurements

In [ ]:
# scaling the data
min_max = MinMaxScaler()
scaled_data = min_max.fit_transform(master_numeric)

#make a dataframe
master_numeric_scaled = pd.DataFrame(data=scaled_data, columns=master_numeric.columns)

#check variance of scaled numeric columns
master_numeric_scaled.var()

In [ ]:
# Instantiate  VarianceThreshold and set threshold for the variance
vt = VarianceThreshold(threshold=0.0005)

# Fit the data and calculate the variances per column
vt.fit(master_numeric_scaled)

In [ ]:
#the variances per column
column_variances = vt.variances_

# Plot including the threshold
plt.figure(figsize=(60,60))
plt.barh(np.flip(master_numeric_scaled.columns), np.flip(column_variances))
plt.xlabel('Variance', size=40)
plt.ylabel('Feature', size=40)
plt.axvline(0.0005, color='red', linestyle='--')
plt.xticks(size=40)
plt.yticks(size=40)
plt.xlim(0,0.1)
plt.show()

Our outcome variable (`is_functioning`) has high variance, which is a very good sign.

The technology of the water point, the installation year and the management entity of the water point have especially high variances.

On the other hand, cluster size, served population, local population, fatalities and pressure all have low variances.

Given the analysis on correlation and variances we choose to drop:

- `cluster_size` due to its low variance and weak correlation with the outcome variable

- `served_population`, `local_population` and `pressure` as they have low variances and are captured by `perc_local_served` created by ourselves.

- `total_fatalities_adm4` because of its variance, low correlation and multicollinearity with `total_events_adm4`.

In [ ]:
# #drop columns
master_df_clean=master_df_clean.drop(columns=['cluster_size', 'served_population', 'local_population', 'pressure', 'total_fatalities_adm4'])

# #check
master_df_clean.columns

# Final feature selection

Finally, we get rid of columns which we feel represent redundant information access to piped water and borehole is very similar to what we are trying to predict. We are wary this might introduce bias in our estimators and we decide to discard these two columns.

In [ ]:
# #drop columns
master_df_clean=master_df_clean.drop(columns=['perc_hh_piped_water', 'perc_hh_borehole'])

# #check
master_df_clean.columns


We have dropped a few columns which were not correlated with our outcome variable, had low variances or had high multicollienarity metrics with other explanatory variables. Our dataset is now ready to be used in models.

In [ ]:
#export new dataset
master_df_clean.to_csv(filepath + 'master_modelling_df.csv')

In [ ]:
#data dictionary
Image("/Users/thomasadler/Desktop/futuristic-platipus/data_dictionary/5-Modelling-Data-Dictionary.png")